In [1]:
import pandas as pd
import numpy as np

# Create date range (12 days)
dates = pd.date_range(start="2024-01-01", periods=12, freq="D")

# Tickers
tickers = ["GOOG", "MSFT"]

# Create cartesian product of dates and tickers
stocks = pd.DataFrame(
    [(date, ticker) for date in dates for ticker in tickers],
    columns=["date", "ticker"]
)

# Add price and volume data (example realistic values)
np.random.seed(42)

stocks["open"] = np.where(
    stocks["ticker"] == "GOOG",
    np.random.uniform(135, 145, len(stocks)),
    np.random.uniform(360, 380, len(stocks))
).round(2)

stocks["close"] = (stocks["open"] + np.random.uniform(-2, 2, len(stocks))).round(2)

stocks["volume"] = np.where(
    stocks["ticker"] == "GOOG",
    np.random.randint(1_800_000, 2_200_000, len(stocks)),
    np.random.randint(2_000_000, 2_400_000, len(stocks))
)

# Final result
stocks


,date,ticker,open,close,volume
0,2024-01-01,GOOG,138.75,138.94,1924375
1,2024-01-01,MSFT,375.70,374.44,2273109
2,2024-01-02,GOOG,142.32,144.20,1936330
3,2024-01-02,MSFT,370.28,371.38,2370210
4,2024-01-03,GOOG,136.56,138.32,1964231
5,2024-01-03,MSFT,360.93,362.51,2375396
6,2024-01-04,GOOG,135.58,135.97,1861858
7,2024-01-04,MSFT,363.41,365.10,2158338
8,2024-01-05,GOOG,141.01,139.36,1812666
9,2024-01-05,MSFT,378.98,377.76,2214020
